<a href="https://colab.research.google.com/github/shoxruxmirzoazamjonov/-calc/blob/main/%D0%94%D0%BE%D0%B1%D1%80%D0%BE_%D0%BF%D0%BE%D0%B6%D0%B0%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B2_Colab!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Установка всех необходимых библиотек
!apt-get update -qq
!apt-get install -y -qq python3-tk xvfb
!pip install pyinstaller pillow -q

print("✅ Все библиотеки установлены!")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package tk8.6-blt2.5.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../tk8.6-blt2.5_2.5.3+dfsg-4.1build2_amd64.deb ...
Unpacking tk8.6-blt2.5 (2.5.3+dfsg-4.1build2) ...
Selecting previously unselected package blt.
Preparing to unpack .../blt_2.5.3+dfsg-4.1build2_amd64.deb ...
Unpacking blt (2.5.3+dfsg-4.1build2) ...
Selecting previously unselected package python3-tk:amd64.
Preparing to unpack .../python3-tk_3.10.8-1~22.04_amd64.deb ...
Unpacking python3-tk:amd64 (3.10.8-1~22.04) ...
Setting up tk8.6-blt2.5 (2.5.3+dfsg-4.1build2) ...
Setting up blt (2.5.3+dfsg-4.1build2) ...
Setting up python3-tk:amd64 (3.10.8-1~22.04) ...
Processing triggers for libc-bin (2.35-0ubuntu3.11) ...
/sbin/ldconfig.real: /usr/local/lib/libumf.so

In [4]:
%%writefile calculator.py
import tkinter as tk
from tkinter import font

class Calculator:
    def __init__(self, root):
        self.root = root
        self.root.title("Калькулятор v1.0")
        self.root.geometry("400x550")
        self.root.configure(bg='#2E2E2E')
        self.root.resizable(False, False)

        # Переменные
        self.current_input = '0'
        self.previous_value = None
        self.current_operation = None
        self.reset_next_input = False

        # Создаем шрифты
        self.display_font = font.Font(family='Arial', size=36, weight='bold')
        self.button_font = font.Font(family='Arial', size=20)

        self.setup_ui()

    def setup_ui(self):
        # Поле вывода
        self.display_var = tk.StringVar(value='0')

        display_frame = tk.Frame(self.root, bg='#2E2E2E')
        display_frame.pack(fill=tk.X, padx=20, pady=(30, 20))

        display = tk.Entry(
            display_frame,
            textvariable=self.display_var,
            font=self.display_font,
            justify='right',
            bd=0,
            bg='#1C1C1C',
            fg='#FFFFFF',
            insertwidth=0,
            readonlybackground='#1C1C1C',
            state='readonly'
        )
        display.pack(fill=tk.X, ipady=20)

        # Фрейм для кнопок
        buttons_frame = tk.Frame(self.root, bg='#2E2E2E')
        buttons_frame.pack(expand=True, fill=tk.BOTH, padx=20, pady=(0, 20))

        # Конфигурация кнопок
        buttons = [
            ('C', 0, 0, '#FF9500'), ('⌫', 0, 1, '#FF9500'), ('%', 0, 2, '#FF9500'), ('/', 0, 3, '#FF9500'),
            ('7', 1, 0, '#505050'), ('8', 1, 1, '#505050'), ('9', 1, 2, '#505050'), ('*', 1, 3, '#FF9500'),
            ('4', 2, 0, '#505050'), ('5', 2, 1, '#505050'), ('6', 2, 2, '#505050'), ('-', 2, 3, '#FF9500'),
            ('1', 3, 0, '#505050'), ('2', 3, 1, '#505050'), ('3', 3, 2, '#505050'), ('+', 3, 3, '#FF9500'),
            ('±', 4, 0, '#505050'), ('0', 4, 1, '#505050'), ('.', 4, 2, '#505050'), ('=', 4, 3, '#FF9500'),
        ]

        # Создаем кнопки
        for text, row, col, color in buttons:
            btn = tk.Button(
                buttons_frame,
                text=text,
                font=self.button_font,
                bg=color,
                fg='white',
                activebackground='#666666',
                activeforeground='white',
                bd=0,
                command=lambda t=text: self.button_click(t),
                relief=tk.FLAT
            )
            btn.grid(row=row, column=col, padx=5, pady=5, sticky='nsew', ipadx=10, ipady=15)

        # Настройка растягивания
        for i in range(5):
            buttons_frame.grid_rowconfigure(i, weight=1)
            buttons_frame.grid_columnconfigure(i, weight=1)

    def button_click(self, value):
        if value in '0123456789':
            if self.reset_next_input:
                self.current_input = value
                self.reset_next_input = False
            elif self.current_input == '0':
                self.current_input = value
            else:
                self.current_input += value

        elif value == '.':
            if '.' not in self.current_input:
                self.current_input += '.' if self.current_input else '0.'

        elif value == '±':
            if self.current_input and self.current_input != '0':
                if self.current_input[0] == '-':
                    self.current_input = self.current_input[1:]
                else:
                    self.current_input = '-' + self.current_input

        elif value == 'C':
            self.current_input = '0'
            self.previous_value = None
            self.current_operation = None
            self.reset_next_input = False

        elif value == '⌫':
            if len(self.current_input) > 1:
                self.current_input = self.current_input[:-1]
            else:
                self.current_input = '0'

        elif value == '%':
            try:
                self.current_input = str(float(self.current_input) / 100)
            except:
                self.current_input = '0'

        elif value in '+-*/':
            if self.current_input:
                if self.previous_value is None:
                    self.previous_value = float(self.current_input)
                elif self.current_operation:
                    try:
                        self.previous_value = self.calculate(
                            self.previous_value,
                            float(self.current_input),
                            self.current_operation
                        )
                    except:
                        self.previous_value = None

                self.current_operation = value
                self.reset_next_input = True

        elif value == '=':
            if self.previous_value is not None and self.current_operation and self.current_input:
                try:
                    result = self.calculate(
                        self.previous_value,
                        float(self.current_input),
                        self.current_operation
                    )

                    # Форматируем результат
                    if isinstance(result, float) and result.is_integer():
                        self.current_input = str(int(result))
                    else:
                        self.current_input = str(result)
                        if '.' in self.current_input:
                            self.current_input = self.current_input.rstrip('0').rstrip('.')

                    self.previous_value = None
                    self.current_operation = None
                    self.reset_next_input = True

                except ZeroDivisionError:
                    self.current_input = 'Ошибка: /0'
                    self.previous_value = None
                    self.current_operation = None
                except:
                    self.current_input = 'Ошибка'
                    self.previous_value = None
                    self.current_operation = None

        self.display_var.set(self.current_input)

    def calculate(self, a, b, operation):
        if operation == '+':
            return a + b
        elif operation == '-':
            return a - b
        elif operation == '*':
            return a * b
        elif operation == '/':
            return a / b

def main():
    root = tk.Tk()
    app = Calculator(root)
    root.mainloop()

if __name__ == "__main__":
    main()

Writing calculator.py


In [5]:
%%writefile create_icon.py
from PIL import Image, ImageDraw

# Создаем иконку калькулятора
img = Image.new('RGB', (256, 256), color='#2E2E2E')
draw = ImageDraw.Draw(img)

# Рисуем корпус калькулятора
draw.rectangle([50, 50, 206, 206], fill='#1C1C1C', outline='#FF9500', width=5)

# Рисуем дисплей
draw.rectangle([60, 60, 196, 100], fill='#505050', outline='#666666')

# Рисуем кнопки
for i in range(4):
    for j in range(4):
        x = 65 + i * 35
        y = 110 + j * 35
        draw.rectangle([x, y, x+25, y+25], fill='#FF9500', outline='#666666')

draw.rectangle([65, 250, 190, 195], fill='#505050', outline='#666666')

img.save('icon.ico', format='ICO', sizes=[(256, 256), (128, 128), (64, 64), (32, 32), (16, 16)])

print("✅ Иконка создана: icon.ico")

Writing create_icon.py


In [6]:
!python create_icon.py
print("✅ Иконка сохранена в текущей директории")

Traceback (most recent call last):
  File "/content/create_icon.py", line 20, in <module>
    draw.rectangle([65, 250, 190, 195], fill='#505050', outline='#666666')
  File "/usr/local/lib/python3.12/dist-packages/PIL/ImageDraw.py", line 396, in rectangle
    self.draw.draw_rectangle(xy, fill_ink, 1)
ValueError: y1 must be greater than or equal to y0
✅ Иконка сохранена в текущей директории


In [7]:
# Компилируем в EXE
!pyinstaller --onefile --windowed --icon=icon.ico --name="Calculator" --clean calculator.py

print("\n" + "="*50)
print("✅ EXE файл создан успешно!")
print("="*50)

75 INFO: PyInstaller: 6.17.0, contrib hooks: 2025.11
76 INFO: Python: 3.12.12
77 INFO: Platform: Linux-6.6.105+-x86_64-with-glibc2.35
77 INFO: Python environment: /usr
78 INFO: wrote /content/Calculator.spec
79 INFO: Removing temporary files and cleaning cache in /root/.cache/pyinstaller
80 INFO: Module search paths (PYTHONPATH):
['/env/python',
 '/usr/lib/python312.zip',
 '/usr/lib/python3.12',
 '/usr/lib/python3.12/lib-dynload',
 '/usr/local/lib/python3.12/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/content']
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
847 INFO: checking Analysis
847 INFO: Building Analysis because Analysis-00.toc is non existent
847 INFO: Looking for Python shared library...
859 INFO: Using Python shared library: /lib/x86_64-linux-gnu/libpython3.12.so.1.0
860 INFO: Running Analysis Analysis-00.toc
860 INFO: Target bytecode optimization level: 0
860 INFO: Initializing module dependency gr

In [8]:
import os

print("Содержимое папки dist/ после сборки:")
!ls -la dist/

print("\nРазмер EXE файла:")
!du -h dist/Calculator 2>/dev/null || echo "На Linux используем stat" && stat -c%s dist/Calculator | awk '{print $1/1024/1024 " MB"}'

print("\n" + "="*50)
print("🎉 Готово! Ваш калькулятор собран в EXE!")
print("="*50)

Содержимое папки dist/ после сборки:
total 11164
drwxr-xr-x 2 root root     4096 Jan  7 08:09 .
drwxr-xr-x 1 root root     4096 Jan  7 08:09 ..
-rwxr-xr-x 1 root root 11422384 Jan  7 08:09 Calculator

Размер EXE файла:
11M	dist/Calculator
10.8932 MB

🎉 Готово! Ваш калькулятор собран в EXE!


In [9]:
from google.colab import files

# Убедимся, что файл существует
if os.path.exists('dist/Calculator'):
    print("Скачиваем Calculator.exe...")

    # Для Windows переименовываем в .exe
    !cp dist/Calculator dist/Calculator.exe

    print("Нажмите на ссылку ниже для скачивания:")
    files.download('dist/Calculator.exe')
elif os.path.exists('dist/Calculator.exe'):
    print("Скачиваем Calculator.exe...")
    print("Нажмите на ссылку ниже для скачивания:")
    files.download('dist/Calculator.exe')
else:
    print("❌ Файл не найден. Проверьте предыдущие шаги.")

Скачиваем Calculator.exe...
Нажмите на ссылку ниже для скачивания:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>